Importing all the necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten 
from tensorflow.keras.layers import Conv2D,MaxPooling2D 

import scipy.io 
from scipy.signal import spectrogram
import matplotlib.pyplot as plt
import os
import random
import numpy as np

Function to create the structure of the CNN model 

In [ ]:
def model_init(inputshape):
	model = Sequential() 
	model.add(Conv2D(16,(3,3),activation='relu',input_shape=inputshape))
	model.add(Conv2D(16,(3,3),activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
	model.add(Flatten())
	model.add(Dense(32, activation='relu'))
	model.add(Dropout(0.5))
	model.add(Dense(2, activation='sigmoid'))

	model.compile(loss=tf.keras.losses.binary_crossentropy,
		      optimizer=tf.keras.optimizers.RMSprop(),
		      metrics=['accuracy'])
	return model
	

there are 18 brain wave plots for preictal segment 

Extracting each of the plot and saving them in the form of spectrogram which is visualized under the standard fast fourier transform.

A spectrogram is a visual representation of the spectrum of frequencies of a signal as it varies with time 

This data will be passed on to the CNN Model.

In [12]:

x=[]
y=[] 

types = ['Patient_1_interictal_segment','Patient_1_preictal_segment']
for i,typ in enumerate(types):
	for j in range(18):
		f1='Patient_1/'+typ+'_'+str(j+1).zfill(4)+'.mat' 
		data=scipy.io.loadmat(f1) 
		k = typ.replace('Patient_1_','')+'_' 
		d_array=data[k+str(j+1)][0][0][0] 
		lst=list(range(3000000)) 
		for m in lst[::5000]:
			p_secs=d_array[0][m:m+5000] 
			p_f, p_t, p_Sxx = spectrogram(p_secs, fs=5000, return_onesided=False)
			p_SS = np.log1p(p_Sxx)
			arr=p_SS[:]/np.max(p_SS)
			x.append(arr)
			y.append(i)
 
dataset = list(zip(x,y)) 
random.shuffle(dataset) 
x,y = zip(*dataset)
print(len(x))

21600


Dividing the dataset into test and training

In [ ]:
x_train =np.array(x[:21000]) 
x_test = np.array(x[21000:])
y_train = np.array(y[:21000]) 
y_test = np.array(y[21000:])

x_train = x_train.reshape(x_train.shape[0],256,22,1) 
x_test = x_test.reshape(x_test.shape[0],256,22,1) 
input_shape = (256,22,1)
x_train = x_train.astype('float32') 
x_test = x_test.astype('float32')


y_train = tf.keras.utils.to_categorical(y_train,2) 
y_test = tf.keras.utils.to_categorical(y_test,2) 


Model is  trained on the training set

storing the datapoint using callbacks provided by tensorflow

In [1]:


checkpoint_path='./training/training.ckpt'

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True,verbose=1)

#print(y_train)	
model = model_init(input_shape) 
model.summary()
		
model.fit(x_train,y_train,epochs=50,verbose=1,validation_data=(x_test,y_test),callbacks=[cp_callback]) 
score=model.evaluate(x_test,y_test,verbose=0) 

print('Test loss:',score[0]) 
print('Test Accuracy:',score[1]) 

new_model = model_init(input_shape) 



2023-03-31 00:27:33.325905: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 00:27:33.850941: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/admin1/miniconda3/envs/tf/lib/
2023-03-31 00:27:33.850980: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/admin1/miniconda3/envs/tf/lib/
2023-03-31 00:27:33.850984: W tensorflow/compiler/tf2t

21600
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 20, 16)       160       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 18, 16)       2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 9, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 126, 9, 16)        0         
                                                                 
 flatten (Flatten)           (None, 18144)             0         
                                                                 
 dense (Dense)               (None, 32)                580640    
                                                  

2023-03-31 00:27:42.468672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-31 00:27:42.468721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (admin1-MS-7C92): /proc/driver/nvidia/version does not exist
2023-03-31 00:27:42.469337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
655/657 [============================>.] - ETA: 0s - loss: 0.4809 - accuracy: 0.6691
Epoch 1: saving model to ./training/training.ckpt
657/657 [==============================] - 21s 31ms/step - loss: 0.4809 - accuracy: 0.6692 - val_loss: 0.4763 - val_accuracy: 0.6500
Epoch 2/50
655/657 [============================>.] - ETA: 0s - loss: 0.4687 - accuracy: 0.6853
Epoch 2: saving model to ./training/training.ckpt
657/657 [==============================] - 20s 31ms/step - loss: 0.4685 - accuracy: 0.6854 - val_loss: 0.4780 - val_accuracy: 0.6583
Epoch 3/50
655/657 [============================>.] - ETA: 0s - loss: 0.4634 - accuracy: 0.6959
Epoch 3: saving model to ./training/training.ckpt
657/657 [==============================] - 20s 30ms/step - loss: 0.4635 - accuracy: 0.6959 - val_loss: 0.4678 - val_accuracy: 0.6950
Epoch 4/50
655/657 [============================>.] - ETA: 0s - loss: 0.4580 - accuracy: 0.7058
Epoch 4: saving model to ./training/training.ckpt
657/657 [========